In [1]:
import os
import io
import dask.dataframe as dd
import pandas as pd
from dask.distributed import Client
import seaborn as sns
import coiled
import dask
import glob

In [2]:
#Create a local Dask cluster and connect it to the client. This is useful to follow the computation memory usage and execution time.  
#Requires Dask JupyterLab extension to follow the Dashboard. For more information, please look at: https://docs.dask.org/en/stable/dashboard.html
client = Client(n_workers=4)
client

Client Scheduler: tcp://127.0.0.1:43643 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 36 Memory: 21.47 GB


In [3]:
#Set the working directory. 
os.chdir("/data/leuven/351/vsc35102/scratch/Full_Data_Set/")

In [ ]:
#Fetch the data from an online location
#gdown.download_folder("https://drive.google.com/drive/folders/1HT-ctj8Aj6qcVMZYBxi3YM4XC9fbFjSN?usp=share_link", quiet=False)

In [ ]:
#Define the function to covert the .csv files into .parquet files, optimized for a fast runtime.

class CSVtoParquetConverter:
    def __init__(self, base_folder):
        self.base_folder = base_folder

    def convert_csv_to_parquet(self):
        months = ['Jan', 'Feb', 'March', 'April', 'May', 'June', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
        for month in months:
            folder_path = os.path.join(self.base_folder, month)
            csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

            for csv_file in csv_files:
                csv_path = os.path.join(folder_path, csv_file)
                df = dd.read_csv(csv_path, sep=";")

                parquet_file = os.path.splitext(csv_file)[0] + '_parquet'
                parquet_path = os.path.join(folder_path, parquet_file)

                df.to_parquet(parquet_path, engine='pyarrow', compression="snappy")
                print(f"Converted {csv_file} to {parquet_file}")

base_folder = '/data/leuven/351/vsc35102/scratch/Full_Data_Set/'
converter = CSVtoParquetConverter(base_folder)
converter.convert_csv_to_parquet()


# Runtime: 2m 6.8 sec for all files on average.


In [22]:
# Process .parquet files into Dask dataframe, subsample data into 10min intervals and return pandas dataframe. 
#Note that this chunk was not optimized in oop fashion to boost the performance.


def process_data_by_month():
    os.chdir("/data/leuven/351/vsc35102/scratch/Full_Data_Set/")

    months = ['Jan', 'Feb', 'March', 'April', 'May', 'June', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    base_folder = '/data/leuven/351/vsc35102/scratch/Full_Data_Set/'
    output_folder = '/data/leuven/351/vsc35102/scratch/Full_Data_Set/output/'
    os.makedirs(output_folder, exist_ok=True)  # Create output folder (in case it done not yet exisit).

    dfs = []  # List to store the individual Pandas dataframes. 

    for month in months:
        flat_list = glob.glob(os.path.join(base_folder, month, '*_parquet/*.parquet'))
        ddf = dd.read_parquet(flat_list, columns=["#object_id", "description", "result_timestamp", "laeq"]) #Read only columns that are going to be used in the model.

        # Convert to the desired data types to boost the performance. Note that object type are very computationaly expensive and it is recommended to specify the corresponding datatype for each column.
        ddf = ddf.astype({'description': 'string', 'laeq': 'float'})

        # Set the timestamp into the correct form and set the index to result_timestamp. Note that this is a bottleneck step.
        ddf["result_timestamp"] = dd.to_datetime(ddf["result_timestamp"], dayfirst=True, format="%d/%m/%Y %H:%M:%S.%f")
        ddf_sorted = ddf.set_index('result_timestamp')

        pandas_df = ddf_sorted.compute().reset_index()  # Convert to Pandas DataFrame.

        pandas3 = pandas_df.groupby("description").resample("10T", on="result_timestamp").agg({'laeq': 'mean'}).reset_index()

        dfs.append(pandas3)

    combined_df = pd.concat(dfs)  # Combine all individual  dataframes into a single dataframe.
    return combined_df

# Runtime: 9min 3sec for all files on average.

In [23]:
# Call the function to process the data for each month
table = process_data_by_month()


In [25]:
# View the table.
table.head()

,description,result_timestamp,laeq
0,MP 03: Naamsestraat 62 Taste,2022-01-01 00:00:00,63.267554
1,MP 03: Naamsestraat 62 Taste,2022-01-01 00:10:00,60.254000
2,MP 03: Naamsestraat 62 Taste,2022-01-01 00:20:00,55.143907
3,MP 03: Naamsestraat 62 Taste,2022-01-01 00:30:00,58.230167
4,MP 03: Naamsestraat 62 Taste,2022-01-01 00:40:00,54.220466


In [36]:
class TimeFeatureGenerator:
    def __init__(self):
        pass

    def add_time_features(self, df):
        df['date'] = df['result_timestamp'].dt.date
        df['time'] = df['result_timestamp'].dt.time
        df['hour'] = df['result_timestamp'].dt.hour
        df['weekday'] = df['result_timestamp'].dt.day_name()
        df['month'] = df['result_timestamp'].dt.month
        df['weekend'] = df['weekday'].isin(['Saturday', 'Sunday'])
        return df

time_feature_generator = TimeFeatureGenerator()
time_features = time_feature_generator.add_time_features(table)


In [37]:
#Define holidays that may impact the noise levels

class HolidayDetector:
    def __init__(self):
        self.holidays = self.generate_holidays()

    def generate_holidays(self):
        holiday_ranges = []
        holiday_dates = [
            ('2022-01-01', '2022-01-13'),
            ('2022-02-02', '2022-02-02'),
            ('2022-02-06', '2022-02-13'),
            ('2022-04-02', '2022-04-18'),
            ('2022-05-01', '2022-05-01'),
            ('2022-05-26', '2022-05-26'),
            ('2022-05-28', '2022-06-12'),
            ('2022-07-03', '2022-09-25'),
            ('2022-11-01', '2022-11-02'),
            ('2022-11-11', '2022-11-11'),
            ('2022-12-24', '2022-12-31')
        ]
        for start_date, end_date in holiday_dates:
            holiday_ranges.append(pd.date_range(start=start_date, end=end_date))
        holidays = pd.concat([pd.DataFrame(date_range) for date_range in holiday_ranges], ignore_index=True)
        holidays.columns = ['dates']
        return holidays

    def detect_holidays(self, time_features):
        time_features['holiday'] = time_features['date'].astype(str).isin(self.holidays['dates'].astype(str))
        return time_features
    
holiday_detector = HolidayDetector()
time_features = holiday_detector.detect_holidays(time_features)


In [38]:

class ExamPeriodDetector:
    def __init__(self):
        self.exam_periods = self.generate_exam_periods()

    def generate_exam_periods(self):
        exam_ranges = [
            ('2022-01-14', '2022-02-05'),
            ('2022-06-13', '2022-07-02'),
            ('2022-08-22', '2022-09-10')
        ]
        exam_periods = pd.concat([pd.DataFrame(pd.date_range(start, end)) for start, end in exam_ranges], ignore_index=True)
        exam_periods.columns = ['dates']
        return exam_periods

    def detect_exam_periods(self, time_features):
        time_features['exam_period'] = time_features['date'].astype(str).isin(self.exam_periods['dates'].astype(str))
        return time_features


exam_period_detector = ExamPeriodDetector()
time_features = exam_period_detector.detect_exam_periods(time_features)


In [40]:
# Localize the datetime object to the Belgian timezone
# This keeps in mind the change from CET to CEST in spring and CEST to CET in autumn
time_features['result_timestamp'] = pd.to_datetime(time_features['result_timestamp']).dt.tz_localize('CET',ambiguous='NaT',nonexistent='NaT')

time_features = time_features.dropna(subset=['result_timestamp'])


TypeError: Already tz-aware, use tz_convert to convert.

In [43]:
#Save it to .csv file
time_features.to_csv('/data/leuven/351/vsc35102/scratch/Full_Data_Set/time_features.csv', index=False)

In [44]:
#Read in the weather data 

def read_multiple_csv(folder_path, columns_to_select):
    dfs = []  # List to store individual DataFrames

    for file in os.listdir(folder_path):
        if file.endswith('.csv'):
            file_path = os.path.join(folder_path, file)
            df = pd.read_csv(file_path, usecols=columns_to_select, parse_dates=['DATEUTC'])
            dfs.append(df)

    combined_df = pd.concat(dfs, ignore_index=True)
    return combined_df

folder_path = '/data/leuven/351/vsc35102/scratch/Full_Data_Set/weather'
columns_to_select = ['ID','DATEUTC', 'LC_HUMIDITY', 'LC_DWPTEMP', 'LC_n', 'LC_RAD', 'LC_RAININ', 'LC_DAILYRAIN', 'LC_WINDDIR', 'LC_WINDSPEED', 'LC_RAD60', 'LC_TEMP_QCL0', 'LC_TEMP_QCL1', 'LC_TEMP_QCL2', 'LC_TEMP_QCL3']

weather = read_multiple_csv(folder_path, columns_to_select)




In [45]:
weather.head()

,DATEUTC,ID,LC_HUMIDITY,LC_DWPTEMP,LC_n,LC_RAD,LC_RAININ,LC_DAILYRAIN,LC_WINDDIR,LC_WINDSPEED,LC_RAD60,LC_TEMP_QCL0,LC_TEMP_QCL1,LC_TEMP_QCL2,LC_TEMP_QCL3
0,2022-01-01 00:10:00,LC-002,92.0,11.78,38.0,0.0,0.0,0.0,-169.0,0.43,0.0,13.11,13.11,13.0515,13.048027
1,2022-01-01 00:20:00,LC-002,92.0,11.73,37.0,0.0,0.0,0.0,-170.0,0.33,0.0,13.01,13.01,12.9515,12.985849
2,2022-01-01 00:30:00,LC-002,92.0,11.73,38.0,0.0,0.0,0.0,-167.0,0.46,0.0,13.00,13.00,12.9415,12.950322
3,2022-01-01 00:40:00,LC-002,92.0,11.72,37.0,0.0,0.0,0.0,-160.0,0.52,0.0,13.00,13.00,12.9415,12.949550
4,2022-01-01 00:50:00,LC-002,92.0,11.72,38.0,0.0,0.0,0.0,-166.0,0.51,0.0,13.00,13.00,12.9415,12.952268


In [ ]:
# Select the relevant location.
weather105_location = weather[weather['ID'] == 'LC-105']

# Modify the timestamp. 
weather105_location.rename(columns={'DATEUTC':'result_timestamp'},inplace=True)
weather105_location['result_timestamp'] = pd.to_datetime(weather105_location['result_timestamp'],format="%Y-%m-%d %H:%M:%S")

#Add the UTC timezone information to the result_timestamp
weather105_location['result_timestamp'] = pd.to_datetime(weather105_location['result_timestamp']).dt.tz_localize('UTC')

#Change weather data to CET timezone
weather105_location['result_timestamp'] = pd.to_datetime(weather105_location['result_timestamp']).dt.tz_convert('CET')

In [49]:
# Merge noise and weather data.
full_dataframe = pd.merge(time_features, weather105_location, on='result_timestamp')

In [51]:
# Remove rows with only weather data, but no noise data.
full_dataframe = full_dataframe[full_dataframe.isnull()['laeq']==False]


In [55]:
#Save the data 
full_dataframe.to_csv('/data/leuven/351/vsc35102/scratch/Full_Data_Set/Project_dataframe.csv', sep=';', index=False)